In [1]:
import pickle
from pclpy import pcl as pcl2
import pcl
import pcl.pcl_visualization
import time
import numpy as np
from open3d import *
from open3d import JVisualizer
import copy

In [1]:
def depthFrametoPC(depth, rgb, cameraIntrinsics, poseMat):
    #starttime = time.time()
    [height,width] = np.array(depth.shape)
    #[height, width] = [depthFrame.get_height(), depthFrame.get_width()]
    #depth = np.asanyarray(depthFrame.get_data())
    #rgb = np.asanyarray(colorFrame.get_data())
    ##time1 = time.time()
    nx = np.linspace(0, width-1, width)
    ny = np.linspace(0, height-1, height)
    u, v = np.meshgrid(nx, ny)
    x = (u.flatten() - cameraIntrinsics['ppx'])/cameraIntrinsics['fx']
    y = (v.flatten() - cameraIntrinsics['ppy'])/cameraIntrinsics['fy']
    z = depth.flatten() / 1000
    x = np.multiply(x,z)
    y = np.multiply(y,z)
    ##time2 = time.time()
    #x = x[np.nonzero(z)]
    #y = y[np.nonzero(z)]
    #z = z[np.nonzero(z)]

    rgbB = rgb[:,:,0].flatten().astype(int)
    rgbG = rgb[:,:,1].flatten().astype(int)
    rgbR = rgb[:,:,2].flatten().astype(int)
    rgbPC = rgbR<<16|rgbG<<8|rgbB
    ##time3=time.time()
    points = np.asanyarray([x,y,z])

    n = points.shape[1] 
    points_ = np.vstack((points, np.ones((1,n))))
    points_trans_ = np.matmul(poseMat, points_)
    points_transformed = np.true_divide(points_trans_[:3,:], points_trans_[[-1], :])
    ##time4=time.time()
    allPoints = np.asanyarray([points_transformed[0,:],points_transformed[1,:],points_transformed[2,:],rgbR/256,rgbG/256,rgbB/256]).T
    ##times = np.array([time1,time2,time3,time4])-starttime
    ##print(times)
    return allPoints



In [18]:

filename = 'loco5foot-1-25.pickle'
file = open(filename, 'rb')
#dataRead = []
#cloud1 = pcl.PointCloud_PointXYZRGBA()
#cloud2 = pcl.PointCloud_PointXYZRGBA()
#visual = pcl.pcl_visualization.CloudViewing()
#time.sleep(2)
for i in range(1,60):
    try:
        frame = pickle.load(file) #this loads each frame in one at a time, this is where we can process each frame with the transformation matrix
        #cloud = pcl.PointCloud_PointXYZRGBA()
        #cloud2 = pcl.PointCloud_PointXYZRGBA()
        allPoints = np.empty((0,4))
        #for (serial, [poseMat, rmsdValue]) in self.devicesTransformation.items():
        pointClouds = []
        j=0
        for camera,deviceData in frame.items():
            #cloud = pcl.PointCloud_PointXYZRGBA()
            pcd = PointCloud()
            depthFrame = deviceData['depth']
            colorFrame = deviceData['color']
            cameraIntrinsics = deviceData['intrinsics']
            poseMat = deviceData['poseMat']
            points = depthFrametoPC(depthFrame, colorFrame, cameraIntrinsics, poseMat)
            pcd.points=Vector3dVector(points[:,0:3])
            pcd.colors=Vector3dVector(points[:,3:6])
            pointClouds = np.append(pointClouds,pcd)#cloud.from_array(points.astype('float32'))
            #visual.ShowColorACloud(cloud)
            j += 1
            #print(j)
            #allPoints = np.append(allPoints, points,axis=0)
        #cloud.from_array(allPoints.astype('float32'))
        i+=1

    except:
        pass
file.close()

In [20]:

draw_geometries([pointClouds[i] for i in [0,2,5]])
draw_geometries([pointClouds[i] for i in [1,3,4]])
#source = pointClouds[0]
#target = pointClouds[1]

In [6]:
draw_geometries([pointClouds[i] for i in [0,1,2,3,4,5]])


In [5]:

def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    draw_geometries([source_temp, target])



In [8]:
newPCs = copy.deepcopy(pointClouds)
for i in range(1,6):
    newPCs[i]=colorRegist(pointClouds[i], pointClouds[0])
draw_geometries([i for i in newPCs])

3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness = 0.435331, inlier_rmse = 0.021291, and correspondence_set size of 138
Access transformation to get result.
3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness = 0.495798, inlier_rmse = 0.013102, and correspondence_set size of 118
Access transformation to get result.
3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness = 0.547468, inlier_rmse = 0.020111, and correspondence_set size of 173
Access transformation to get result.
3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate nor

In [29]:
# draw initial alignment
current_transformation = np.identity(4)
draw_registration_result_original_color(
        source, target, current_transformation)

# point to plane ICP
current_transformation = np.identity(4);
print("2. Point-to-plane ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.02.")
result_icp = registration_icp(source, target, 0.02,
        current_transformation, TransformationEstimationPointToPlane())
print(result_icp)
draw_registration_result_original_color(
        source, target, result_icp.transformation)
#this doesn't seem to be necessary since it doesn't have any improvement over kabsch algorithm

2. Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.02.
RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.


In [7]:
def colorRegist(source, target):# colored pointcloud registration
    # This is implementation of following paper
    # J. Park, Q.-Y. Zhou, V. Koltun,
    # Colored Point Cloud Registration Revisited, ICCV 2017
    voxel_radius = [ 0.04, 0.02, 0.01 ];
    max_iter = [ 50, 30, 24 ];
    current_transformation = np.identity(4)
    print("3. Colored point cloud registration")
    for scale in range(3):
        iter = max_iter[scale]
        radius = voxel_radius[scale]
        print([iter,radius,scale])

        print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = voxel_down_sample(source, radius)
        target_down = voxel_down_sample(target, radius)

        print("3-2. Estimate normal.")
        estimate_normals(source_down, KDTreeSearchParamHybrid(
                radius = radius * 2, max_nn = 30))
        estimate_normals(target_down, KDTreeSearchParamHybrid(
                radius = radius * 2, max_nn = 30))

        print("3-3. Applying colored point cloud registration")
        result_icp = registration_colored_icp(source_down, target_down,
                radius, current_transformation,
                ICPConvergenceCriteria(relative_fitness = 1e-6,
                relative_rmse = 1e-6, max_iteration = iter))
        current_transformation = result_icp.transformation
        print(result_icp)
        source_temp = copy.deepcopy(source)
        #print(np.asanyarray(source_temp.points))
        source_temp.transform(result_icp.transformation)
        #print(np.asanyarray(source_temp.points))
        return source_temp


In [47]:
pointClouds.

dict_items([(0, PointCloud with 407040 points.), (1, PointCloud with 407040 points.), (2, PointCloud with 407040 points.), (3, PointCloud with 407040 points.), (4, PointCloud with 407040 points.), (5, PointCloud with 407040 points.)])

In [ ]:
cloud0 = pcl.PointCloud()
cloud1 = pcl.PointCloud()
cloud2 = pcl.PointCloud()
cloud3 = pcl.PointCloud()
cloud4 = pcl.PointCloud()
cloud5 = pcl.PointCloud()

cloud0.from_array(pointClouds[0][:,0:3].astype('float32'))
cloud1.from_array(pointClouds[1][:,0:3].astype('float32'))
cloud2.from_array(pointClouds[2][:,0:3].astype('float32'))
cloud3.from_array(pointClouds[3][:,0:3].astype('float32'))
cloud4.from_array(pointClouds[4][:,0:3].astype('float32'))
cloud5.from_array(pointClouds[5][:,0:3].astype('float32'))

In [ ]:
icp1 = cloud0.make_IterativeClosestPoint()
converged, transf, estimate1, fitness=icp1.icp(cloud0, cloud1)

icp2 = cloud0.make_IterativeClosestPoint()
converged, transf, estimate2, fitness=icp2.icp(cloud0, cloud2)

icp3 = cloud0.make_IterativeClosestPoint()
converged, transf, estimate3, fitness=icp3.icp(cloud0, cloud3)

icp4 = cloud0.make_IterativeClosestPoint()
converged, transf, estimate4, fitness=icp4.icp(cloud0, cloud4)

icp5 = cloud0.make_IterativeClosestPoint()
converged, transf, estimate5, fitness=icp5.icp(cloud0, cloud5)

In [ ]:
icp = cloud1.make_IterativeClosestPoint()
icp.icp(cloud1, cloud2)

In [ ]:
visual = pcl.pcl_visualization.CloudViewing()
visual.ShowMonochromeCloud(cloud2)

In [54]:
file.close()